# Bysykkel and Topography: Analyzing Oslo's Urban Bike Patterns
This project explores how Oslo's elevation influences bike-sharing usage using data from the entire year. Enrich station data with elevation, calculate elevation differences and gradients per trip, and analyze usage trends.  
  
Data source: https://oslobysykkel.no/apne-data/historisk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import duckdb
import glob
import requests
import os
from haversine import haversine, Unit
from datetime import datetime as dt
import seaborn as sns
import folium

fig_size = (16,10)

## Step 1 – Load Raw Trip Data into DuckDB

This step reads all monthly CSV files from the `../data/` folder and loads them into a DuckDB database file (`db/bysykkel_2024.duckdb`). If the database doesn't exist, it will be created. 

In [ ]:
# Load all CSVs into DuckDB table 'trips_raw'
con = duckdb.connect("../db/bysykkel_2024.duckdb")

csv_files = glob.glob("../data/??.csv")
for i, file in enumerate(csv_files):
    if i == 0:
        con.execute(f"CREATE OR REPLACE TABLE trips_raw AS SELECT * FROM read_csv_auto('{file}')")
    else:
        con.execute(f"INSERT INTO trips_raw SELECT * FROM read_csv_auto('{file}')")

con.execute("CHECKPOINT")
print("Loaded trips_raw data into DuckDB")

## Step 1.1 - Clean the dataset, explore columns

In [ ]:
trips = con.execute('SELECT * FROM trips_raw').df()
trips.info()

There are no nan values, except for in station descriptions. Remove the station descriptions, they are not useful.

In [ ]:
trips = trips.drop(['start_station_description', 'end_station_description'], axis=1, errors='ignore')

##### 1.1.1 Duration

In [ ]:
plt.figure(figsize=fig_size)
(trips['duration']/60).hist(bins=600)
plt.xlim([0, 60])
plt.xlabel('Ride duration [min]')
plt.ylabel('Number of rides')
plt.title('Histogram for the ride duration');

In [ ]:
trips['duration'].describe()

In [ ]:
# Remove trips that are longer than 2 hours 
trips = trips[trips['duration']<=7200]

##### 1.1.2 started_at


In [ ]:
# Sanity check: Check if ended_at is always after started_at
(trips['ended_at'] >= trips['started_at']).all()

In [ ]:
import matplotlib.dates as mdates

# Group by day
daily = trips.groupby(trips['started_at'].dt.date).size()

# Convert index to datetime (from date)
daily.index = pd.to_datetime(daily.index)

# Plot
plt.figure(figsize=fig_size)
plt.plot(daily.index, daily.values, color='royalblue')

# Format x-axis with months
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.title("Daily Ride Count (Binned by Day)")
plt.xlabel("Month")
plt.ylabel("Number of Rides")
plt.grid(True)
plt.tight_layout()
plt.show()

There are two peaks, one before and one right after the summer vacation in July.

##### Investigate loops  
Check how many rides had identical start and end stations.

In [ ]:
df_loops = trips[trips['start_station_id']==trips['end_station_id']]
len(df_loops)

In [ ]:
plt.figure(figsize=fig_size)
(df_loops['duration']/60).hist(bins=1000)
plt.xlim([0, 5])
plt.xlabel('Ride duration [min]')
plt.ylabel('Number of rides')
plt.title('Distribution of ride duration with identical start and end station');

I want to investigate topology. Do loops even matter here? No. They don’t carry slope info. I will remove the useless ones and tag the rest.  
  
  Trips that started and ended at the same station with a duration under 3 minutes will be removed, as they likely represent cancelled or test rides. Remaining loops will be tagged as is_loop but excluded from slope-based analysis, as their paths and elevation data cannot be meaningfully inferred.

In [ ]:
# Drop short loops under 3 minutes
trips = trips[~((trips['start_station_id'] == trips['end_station_id']) & (trips['duration'] < 180))]

# Tag remaining loops
trips.loc[:,'is_loop'] = trips['start_station_id'] == trips['end_station_id']

In [ ]:
# Plot duration distribution, log scale helps
plt.figure(figsize=fig_size)
sns.histplot(data=trips, x='duration', hue='is_loop', bins=100, log_scale=True)
plt.title("Ride Duration: Loops vs. Non-Loops")
plt.xlabel("Ride Duration [sec]")
plt.ylabel("Count");

In [ ]:
# Save cleaned and tagged trips back to DuckDB
con.register("cleaned_df", trips)
con.execute("CREATE OR REPLACE TABLE trips_v1_cleaned AS SELECT * FROM cleaned_df")
con.unregister("cleaned_df")
con.execute("CHECKPOINT")
print("Cleaned trips saved to DuckDB as 'trips_v1_cleaned'")

## Step 2 – Extract Unique Stations

Build a `stations` table by combining all distinct start and end stations from the `trips_cleaned` table. This gives the full list of physical bike stations to enrich with elevation later.

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE stations AS
SELECT DISTINCT
    station_id,
    station_name,
    lat,
    lon
FROM (
    SELECT
        start_station_id AS station_id,
        start_station_name AS station_name,
        start_station_latitude AS lat,
        start_station_longitude AS lon
    FROM trips_v1_cleaned

    UNION

    SELECT
        end_station_id AS station_id,
        end_station_name AS station_name,
        end_station_latitude AS lat,
        end_station_longitude AS lon
    FROM trips_v1_cleaned
)
ORDER BY station_id
""")

con.execute("CHECKPOINT")
print("Extracted and saved stations table")

## Step 3 - Using open-elevation api to populate the stations table with elevation data

In [ ]:
csv_path = "../db/stations_with_elevation.csv"

def get_elevation(row):
    lat = row['lat']
    lon = row['lon']
    
    response = requests.get(f"https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}")
    data = response.json()
    return data['elevation'][0]

if os.path.exists(csv_path):
    print("Found existing data. Loading from csv.")
    stations = pd.read_csv(csv_path)
    stations = stations.drop_duplicates(subset='station_id')
else:
    print("No csv found. Fetching elevation data from API.")
    stations = con.execute("SELECT * FROM stations").df()
    stations['elevation'] = stations.apply(get_elevation, axis=1)
    stations = stations.drop_duplicates(subset='station_id')
    stations.to_csv("../data/stations_with_elevation.csv", index=False)

print("Elevation received for all stations.")

In [ ]:
# Save elevation data to database
con.register("stations_df", stations)
con.execute("CREATE OR REPLACE TABLE stations AS SELECT * FROM stations_df")
con.unregister("stations_df");

In [ ]:
# Join elevation data onto trips table
con.execute("""
CREATE OR REPLACE TABLE trips_v2_elevation AS
SELECT
    t.*,   
    s_start.elevation AS start_elevation,
    s_end.elevation AS end_elevation,
    s_end.elevation - s_start.elevation AS elevation_diff
FROM trips_v1_cleaned t
JOIN stations s_start ON t.start_station_id = s_start.station_id
JOIN stations s_end ON t.end_station_id = s_end.station_id
""")

con.execute("CHECKPOINT");

In [ ]:
con.execute("SHOW TABLES").df()

In [ ]:
trips = con.execute("SELECT * FROM trips_v2_elevation").df()

In [ ]:
trips['elevation_diff'].hist(bins=50)

In [ ]:
trips['elevation_diff'].median()

In [ ]:
trips['elevation_diff'].skew()

## Step 4 - Compute travel distance and gradient

In [ ]:
def compute_distance(row):
    start = (row['start_station_latitude'], row['start_station_longitude'])
    end = (row['end_station_latitude'], row['end_station_longitude'])
    return haversine(start, end, unit=Unit.METERS)

trips["distance"] = trips.apply(compute_distance, axis=1)

def compute_gradient(row):
    if row['distance'] == 0:
        return np.nan
    return row['elevation_diff'] / row['distance'] * 100

trips["gradient"] = trips.apply(compute_gradient, axis=1)

# Update table with final fields
con.register("stage_df", trips)
con.execute("CREATE OR REPLACE TABLE trips_v3_gradient AS SELECT * FROM stage_df")
con.unregister("stage_df")
con.execute("CHECKPOINT");

In [ ]:
con.execute("SHOW TABLES").df()

In [ ]:
trips = con.execute("FROM trips_v3_gradient SELECT *").df()
trips.head(2)

In [ ]:
trips["gradient"].median()

In [ ]:
trips["gradient"].hist(bins=200)
plt.xlim([-10, 10])

In [ ]:
# Ich glaube sehr viele Leute fahren einfach am Meer entlang. Das ist sicher der Peak bei 0. wenn man den entfernt, dann sieht man vielleicht wie "skewed" es ist. Was ist wenn du die beliebtesten Routen ausfindig machst. Verusche ein Muster herauszufinen. 

In [ ]:
trips['gradient'].skew()

In [ ]:
n_downhill = trips['gradient'][trips['gradient']<0].size
n_uphill = trips['gradient'][trips['gradient']>0].size
total_rides = trips['gradient'].size

net_bias = (n_downhill - n_uphill) / total_rides * 100
relative_increase = (n_downhill - n_uphill) / n_uphill * 100

print(f"Out of all rides, {net_bias:.0f}% more go downhill than uphill.")
print(f"When comparing only uphill and downhill rides, downhill trips are {relative_increase:.0f}% more common than uphill ones.")

- Computed elevation differece has a skew of -0.22 (mildy downhill-biased)
- Elevation difference alone is misleading -> compute gradient
- Found gradient skew is +0.10, but visually looks more negatively biased
- Noticed a spike around 0-small positive gradient -> suspect: flat seaside rides
- Have a sense that total ride time might be a more honest metric than ride count

## Step 5 - Use ride duration and total distance ridden as new metric
Ride count can be misleading.  
  
Trying these alternate targets:  
    - Total ride duration  
    - Total distance ridden  

Will do this for downhill vs uphill. 

In [ ]:
# Create a slope type
con.execute("""
CREATE OR REPLACE TABLE trips_v4_slope AS
SELECT *,
    CASE
        WHEN gradient IS NULL THEN 'unknown'
        WHEN gradient < 0 THEN 'downhill'
        WHEN gradient > 0 THEN 'uphill'
        ELSE 'flat'
    END AS slope_type
FROM trips_v3_gradient
"""
);

In [ ]:
con.execute("SHOW TABLES").df()

In [ ]:
# Histogram of Trip Duration, split by gradient sign

trips = con.execute("SELECT * FROM trips_v4_slope").df()
trips = trips[~trips['is_loop']]

# Plot histogram
plt.figure(figsize=fig_size)
sns.histplot(data=trips, x='duration', hue='slope_type', hue_order=['flat', 'uphill', 'downhill'], bins=50, kde=True, log_scale=True)
plt.title("Trip Duration Distribution by Gradient Direction")
plt.xlabel("Trip Duration (seconds)")
plt.ylabel("Number of Trips");

In [ ]:
# Histogram of Trip Distance, split by gradient sign

# Plot histogram
plt.figure(figsize=(15,10))
sns.histplot(data=trips, x='distance', hue='slope_type', hue_order=['flat', 'uphill', 'downhill'], bins=50, kde=True, log_scale=True)
plt.title("Trip Distance Distribution by Gradient Direction")
plt.xlabel("Trip Distance (meters)")
plt.ylabel("Number of Trips");

In [ ]:
summary = trips.groupby('slope_type').agg({
    'duration': 'sum',
    'distance': 'sum',
    'gradient': 'mean'
})
print(summary)

In [ ]:
# Extract values
duration_down = summary.loc['downhill', 'duration']
duration_up = summary.loc['uphill', 'duration']
distance_down = summary.loc['downhill', 'distance']
distance_up = summary.loc['uphill', 'distance']

# Relative increase (compared to uphill)
duration_diff_pct = (duration_down - duration_up) / duration_up * 100
distance_diff_pct = (distance_down - distance_up) / distance_up * 100

# Net share difference (relative to total)
duration_total = duration_down + duration_up
distance_total = distance_down + distance_up

duration_net_share = (duration_down - duration_up) / duration_total * 100
distance_net_share = (distance_down - distance_up) / distance_total * 100

print(f"Duration:")
print(f"• Downhill trips took {duration_diff_pct:.1f}% more total ride time than uphill trips.")
print(f"• Net share: {duration_net_share:.1f}% more ride time downhill than uphill.\n")

print(f"Distance:")
print(f"• Downhill trips covered {distance_diff_pct:.1f}% more total distance than uphill trips.")
print(f"• Net share: {distance_net_share:.1f}% more distance ridden downhill than uphill.")

In [ ]:
# Average duration and distance per ride, grouped by slope
avg_summary = trips.groupby('slope_type').agg({
    'duration': ['mean', 'count'],
    'distance': 'mean'
})

avg_summary.columns = ['avg_duration', 'ride_count', 'avg_distance']
avg_summary = avg_summary.reset_index()

print("Average Ride Metrics by Slope Type:\n")
print(avg_summary.to_string(index=False))

for _, row in avg_summary.iterrows():
    slope = row['slope_type']
    print(f"• {slope.capitalize()} rides: average duration = {row['avg_duration']:.1f} seconds, "
          f"average distance = {row['avg_distance']:.1f} meters "
          f"({int(row['ride_count'])} rides)")

On average, uphill rides are 11% longer in time, despite covering ~9% less distance than downhill rides. This suggests that uphill rides are not only fewer, but also slower—consistent with the physical challenge of biking uphill. Downhill rides, on the other hand, are both more common and slightly longer in distance, indicating that gravity is a strong motivator for mobility behavior in Oslo.

Interpretation  
	•	More trips go downhill than uphill  
	•	Downhill trips take more time in total, even though each is typically shorter, suggesting they’re more common  
	•	Uphill trips are longer per ride, because it takes more effort  
	•	This confirms that elevation influences mobility patterns  

## Step 6 - Spatial Intelligence

### 6.1 Investigate which stations are importers and exporters

In [ ]:
flux_out = trips.groupby('start_station_id').size().rename('departures')
flux_in = trips.groupby('end_station_id').size().rename('arrivals')

flux = flux_out.to_frame().join(flux_in.to_frame(), how='outer').fillna(0)
flux['net_flux'] = flux['arrivals'] - flux['departures']
flux['total_usage'] = flux['arrivals'] + flux['departures']
flux['departure_share'] = flux['departures'] / flux['total_usage']

flux = stations.join(flux, how="outer", on="station_id").fillna(0)

# Update table with final fields
con.register("stage_df", flux)
con.execute("CREATE OR REPLACE TABLE flux AS SELECT * FROM stage_df")
con.unregister("stage_df")
con.execute("CHECKPOINT");

#### 6.1.1 Elevation Map

In [ ]:
# Create interactive Folium map of Oslo to show elevation and usage of bike stations

oslo_coordinates = [59.9139, 10.7522]

import branca.colormap as cm

elevation_min = stations['elevation'].min()
elevation_max = stations['elevation'].max()
colormap = cm.LinearColormap(
    colors=['blue', 'yellow', 'red'],
    vmin=elevation_min,
    vmax=elevation_max
)

def get_radius(total_usage):
    # Normalize usage
    return np.interp(total_usage,
                     (flux['total_usage'].min(), flux['total_usage'].max()),
                     (4, 17))
    
elevation_map = folium.Map(location=oslo_coordinates, zoom_start=13)

# Print all stations onto the map
for index, row in flux.iterrows():

    popup_content = f"""
    Station: {row['station_name']} <br>
    Elevation: {row['elevation']:.0f}m <br>
    Departures: {row['departures']} <br>
    Arrivals: {row['arrivals']} <br>
    Total Usage: {row['total_usage']}
    """

    # Add marker
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=get_radius(row['total_usage']),
        color=colormap(row['elevation']),
        fill=True,
        fill_opacity=0.8,
        weight=1,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(elevation_map)

elevation_map.save('../outputs/oslo_elevation_map.html')

elevation_map

#### 6.1.2 Flux Map (importer and exporter stations)

In [ ]:
# Create interactive Folium map of Oslo to spatially demonstrate the behaviour where bikes are flowing

oslo_coordinates = [59.9139, 10.7522]

def get_color(flux_value):
    if flux_value > 1000:
        return '#1a9850' #'darkgreen'
    elif flux_value > 0:
        return 'yellowgreen' 
    elif flux_value > -1000:
        return '#fc8d59' #'orange'
    else:
        return '#d73027' #'red'
    
def get_radius(total_usage):
    # Normalize usage
    return np.interp(total_usage,
                     (flux['total_usage'].min(), flux['total_usage'].max()),
                     (2, 17))

flux_map = folium.Map(location=oslo_coordinates, zoom_start=13)

# Print all stations onto the map
for index, row in flux.iterrows():

    popup_content = f"""
    Station: {row['station_name']} <br>
    Elevation: {row['elevation']:.0f}m <br>
    Departures: {row['departures']} <br>
    Arrivals: {row['arrivals']} <br>
    Total Usage: {row['total_usage']} <br>
    Net Flux: {row['net_flux']}
    """
    
    # Add marker
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=get_radius(row['total_usage']),
        color=get_color(row['net_flux']),
        fill=True,
        fill_color=get_color(row['net_flux']),
        fill_opacity=0.8,
        weight=1,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(flux_map)

# ToDo: Fix legend! Does not work

flux_map.save('../outputs/oslo_flux_map.html')


flux_map

In [ ]:
# Visualize sorted net flux and elevation

sorted_flux = flux.sort_values('net_flux')

fig, ax1 = plt.subplots(figsize=(15, 8))
ax1.plot(range(len(sorted_flux)), sorted_flux['net_flux'], 'b-', linewidth=2, label='Net Flux')
ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.7)

# Create secondary axis for elevation
ax2 = ax1.twinx()
scatter = ax2.scatter(range(len(sorted_flux)), 
            sorted_flux['elevation'], 
            c=sorted_flux['elevation'], 
            cmap='YlOrRd', 
            alpha=0.7, 
            s=50, 
            edgecolor='k', 
            linewidth=0.5, 
            label='Elevation')

# Colorbar for elevation
cbar = plt.colorbar(scatter, pad=0.01)
cbar.set_label('Elevation (m)', rotation=270, labelpad=20)

# Set labels and title
ax1.set_xlabel('Stations (Sorted by Net Flux)', fontsize=12)
ax1.set_ylabel('Net Flux (Arrivals - Departures)', color='b', fontsize=12)
plt.title('Station Net Flux vs. Elevation', fontsize=14)
ax1.tick_params(axis='y', colors='blue')

# Add subset of stations to x-axis
step = 5
ax1.set_xticks(range(0, len(sorted_flux), step))
ax1.set_xticklabels(sorted_flux['station_name'].iloc[::step], rotation=90, ha='center');

In [ ]:
flux.plot(kind="scatter", x='elevation', y='net_flux', figsize=fig_size)
plt.show()
plt.savefig('../outputs/scatter_elev_flux.png')
flux[['elevation', 'net_flux']].corr()

### 6.2 Direct Route Analysis: Compute most popular routes and most popular connections

In [ ]:
trips = con.execute('SELECT * FROM trips_v4_slope').df()

In [ ]:
# Directed routes
trips["route"] = trips["start_station_id"].astype(str) + "->" + trips["end_station_id"].astype(str)
trips['route'] = trips['route'].astype('category')


# Unidirectional route (connection)
trips["connection"] = trips.apply(
    lambda row: f"{min(row['start_station_id'], row['end_station_id'])}<->{max(row['start_station_id'], row['end_station_id'])}",
    axis=1
)
trips['connection'] = trips['connection'].astype('category')

In [ ]:
# Database getting too large. Dropping old tables
tables_to_drop = ['trips_v1_cleaned', 'trips_v2_elevation', 'trips_v3_gradient']
for table in tables_to_drop:
    con.execute(f"DROP TABLE IF EXISTS {table}")

con.execute("VACUUM")

# # Save final trips table
con.register("stage_df", trips)
con.execute("CREATE OR REPLACE TABLE trips_v5_routes AS SELECT * FROM stage_df")
con.unregister("stage_df")
con.execute("CHECKPOINT")
con.execute("SHOW TABLES").df()

#### 6.2.1 Compute stats and tables

In [ ]:
n_routes = 30
n_connections = 100

popular_routes = trips.groupby('route').size().reset_index(name='count').sort_values(by='count', ascending=False)
popular_connections = trips.groupby('connection').size().reset_index(name='count').sort_values(by='count', ascending=False)

top_routes = popular_routes.reset_index(drop=True).head(n_routes)
top_connections = popular_connections.reset_index(drop=True).head(n_connections)

In [ ]:
# Enrich top routes and connections with station info
station_lookup = stations.set_index('station_id')

def get_station_info(row, column, delimiter):
    start_id, end_id = row[column].split(delimiter)

    start_station = station_lookup.loc[int(start_id), 'station_name']
    end_station = station_lookup.loc[int(end_id), 'station_name']

    start_elevation = station_lookup.loc[int(start_id), 'elevation']
    end_elevation = station_lookup.loc[int(end_id), 'elevation']

    start_lat = station_lookup.loc[int(start_id), 'lat']
    end_lat = station_lookup.loc[int(end_id), 'lat']

    start_lon = station_lookup.loc[int(start_id), 'lon']
    end_lon = station_lookup.loc[int(end_id), 'lon']

    return pd.Series({'start_station_id': start_id,
                      'end_station_id': end_id,
                      'start_station': start_station,
                      'end_station': end_station,
                      'start_elevation': start_elevation,
                      'end_elevation': end_elevation,
                      'start_lat': start_lat,
                      'end_lat': end_lat,
                      'start_lon': start_lon,
                      'end_lon': end_lon})

route_info = top_routes.apply(lambda row: get_station_info(row, 'route', '->'), axis=1)
top_routes = pd.concat([top_routes, route_info], axis=1)

connection_info = top_connections.apply(lambda row: get_station_info(row, 'connection', '<->'), axis=1)
top_connections = pd.concat([top_connections, connection_info], axis=1)


#### 6.2.3 Visualize with Folium

In [ ]:
# Create interactive Folium map of Oslo to show most popular bike routes

oslo_coordinates = [59.9139, 10.7522]

import branca.colormap as cm

color_scale = cm.LinearColormap(['lightblue', 'blue', 'darkblue'],
                                 vmin=top_connections['count'].min(),
                                 vmax=top_connections['count'].max())

connection_map = folium.Map(location=oslo_coordinates, zoom_start=13, tiles='CartoDB positron')

def get_weight(count):
    return np.interp(count,
                    (top_connections['count'].min(), top_connections['count'].max()),
                    (2, 20))



# Print all bike routes
for index, row in top_connections.iterrows():
    
    popup_content = f"""
    {row['start_station']} → {row['end_station']}<br>
    Count: {row['count']}
    """

    folium.PolyLine(
        locations=[[row['start_lat'], row['start_lon']],
                   [row['end_lat'], row['end_lon']]],
                   weight=get_weight(row['count']),
                   opacity=0.5,
                   color=color_scale(row['count']),
                   popup=folium.Popup(popup_content, max_width=300),
                    ).add_to(connection_map)
    
for index, row in stations.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        color='grey',
        radius=2,
        fill=True,
        fill_opacity=0.8,
        weight=2,
        ).add_to(connection_map)
    
connection_map

Here it's visible what the most important routes are. But hitting the ceiling of what this approach can do, will get chaotic very soon. Need to find pattern, which live on a higher level. Need to cluster. 

### 6.3 Cluster Analysis (DBscan)

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

coordinates = stations[['lat', 'lon']]

# Normalize coordinates such that latitude and longitude use the same scale
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coordinates)

# Perform DBSCAN
db = DBSCAN(eps=0.14, min_samples=2).fit(coords_scaled)
stations['cluster_id'] = db.labels_

In [ ]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Assign colors
unique_clusters = sorted(stations['cluster_id'].unique())
color_map = {cluster: mcolors.to_hex(cm.tab20(i % 20)) for i, cluster in enumerate(unique_clusters)}

# Create map
cluster_map = folium.Map(location=oslo_coordinates, zoom_start=13,  tiles='CartoDB positron')

for _, row in stations.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color=color_map[row['cluster_id']] if row['cluster_id'] != -1 else 'gray',
        fill=True,
        fill_opacity=0.7,
        popup=f"Station: {row['station_name']}<br>Cluster: {row['cluster_id']}"
    ).add_to(cluster_map)
cluster_map


In [ ]:
# Compute cluster coordinates (centroids)
clusters = stations[stations['cluster_id']>=0].groupby('cluster_id').agg({'lat': 'mean', 'lon': 'mean', 'elevation': 'mean', 'station_name': 'count'})
clusters = clusters.reset_index()
clusters.rename(columns={'lat': 'cluster_lat', 'lon': 'cluster_lon', 'elevation': 'cluster_elevation', 'station_name': 'cluster_size'}, inplace=True)

# Enrich top_connections with start_cluster_id and end_cluster_id
cluster_dict = stations.set_index('station_id')['cluster_id'].to_dict()

top_connections['start_cluster'] = top_connections['start_station_id'].astype(int).map(cluster_dict)
top_connections['end_cluster'] = top_connections['end_station_id'].astype(int).map(cluster_dict)

# Drop rides where both start and end station are cluster noise
top_connections = top_connections[~((top_connections['start_cluster']==-1) & (top_connections['end_cluster']==-1))]

# Add centroid coordinates to top connections
top_connections = top_connections.merge(
    clusters[['cluster_id', 'cluster_lat', 'cluster_lon']].add_prefix('start_'),
    left_on='start_cluster',
    right_on='start_cluster_id',
    how='left'
    )

top_connections = top_connections.merge(
    clusters[['cluster_id', 'cluster_lat', 'cluster_lon']].add_prefix('end_'),
    left_on='end_cluster',
    right_on='end_cluster_id',
    how='left'
    )

# Treat noise clusters 
mask = top_connections['start_cluster'] == -1
top_connections.loc[mask, 'start_cluster_lat'] = top_connections.loc[mask, 'start_lat']
top_connections.loc[mask, 'start_cluster_lon'] = top_connections.loc[mask, 'start_lon']
mask = top_connections['end_cluster'] == -1
top_connections.loc[mask, 'end_cluster_lat'] = top_connections.loc[mask, 'end_lat']
top_connections.loc[mask, 'end_cluster_lon'] = top_connections.loc[mask, 'end_lon']
top_connections.drop(columns=['start_cluster_id', 'end_cluster_id'], inplace=True)

In [ ]:
# Compute top connections between clusters
top_connections['cluster_a'] = top_connections[['start_cluster', 'end_cluster']].min(axis=1)
top_connections['cluster_b'] = top_connections[['start_cluster', 'end_cluster']].max(axis=1)
top_cluster_connections = top_connections.groupby(['cluster_a', 'cluster_b']).agg({
    'count': 'sum',
}).reset_index()

# Add cluster info (lat, lon) to top cluster connections
top_cluster_connections = top_cluster_connections.merge(
    clusters[['cluster_id', 'cluster_lat', 'cluster_lon']],
    left_on='cluster_a',
    right_on='cluster_id',
    how='left'
).rename(columns={
    'cluster_lat': 'start_lat',
    'cluster_lon': 'start_lon'
}).drop(columns='cluster_id')

top_cluster_connections = top_cluster_connections.merge(
    clusters[['cluster_id', 'cluster_lat', 'cluster_lon']],
    left_on='cluster_b',
    right_on='cluster_id',
    how='left'
).rename(columns={
    'cluster_lat': 'end_lat',
    'cluster_lon': 'end_lon'
}).drop(columns='cluster_id')

In [ ]:
# Fix missing coordinates for noise cluster (-1) cases by looking up from top_connections
def get_cluster_coordinates(row, connections_df):
    cluster_a = row['cluster_a']
    cluster_b = row['cluster_b']

    matching_rows = connections_df[
        ((connections_df['start_cluster'] == cluster_a) & (connections_df['end_cluster'] == cluster_b)) |
        ((connections_df['start_cluster'] == cluster_b) & (connections_df['end_cluster'] == cluster_a))
    ]

    if matching_rows.empty:
        return pd.Series({'start_lat': None, 'start_lon': None, 'end_lat': None, 'end_lon': None})

    sample = matching_rows.iloc[0]

    # Resolve A
    if cluster_a == -1:
        a_lat = sample['start_lat'] if sample['start_cluster'] == cluster_a else sample['end_lat']
        a_lon = sample['start_lon'] if sample['start_cluster'] == cluster_a else sample['end_lon']
    else:
        a_lat = clusters[clusters['cluster_id'] == cluster_a]['cluster_lat'].values[0]
        a_lon = clusters[clusters['cluster_id'] == cluster_a]['cluster_lon'].values[0]

    # Resolve B
    if cluster_b == -1:
        b_lat = sample['start_lat'] if sample['start_cluster'] == cluster_b else sample['end_lat']
        b_lon = sample['start_lon'] if sample['start_cluster'] == cluster_b else sample['end_lon']
    else:
        b_lat = clusters[clusters['cluster_id'] == cluster_b]['cluster_lat'].values[0]
        b_lon = clusters[clusters['cluster_id'] == cluster_b]['cluster_lon'].values[0]

    return pd.Series({
        'start_lat': a_lat,
        'start_lon': a_lon,
        'end_lat': b_lat,
        'end_lon': b_lon
    })

# Apply to fix missing coordinates
fixed_coords = top_cluster_connections.apply(lambda row: get_cluster_coordinates(row, top_connections), axis=1)

# Merge back
top_cluster_connections[['start_lat', 'start_lon', 'end_lat', 'end_lon']] = fixed_coords

In [ ]:
top_cluster_connections

In [ ]:
# Create interactive Folium map of Oslo to show flow between clusters

oslo_coordinates = [59.9139, 10.7522]

cluster_connection_map = folium.Map(location=oslo_coordinates, zoom_start=13,  tiles='CartoDB positron')

import branca.colormap as cm

color_scale = cm.LinearColormap(['lightblue', 'blue', 'darkblue'],
                                 vmin=top_cluster_connections['count'].min(),
                                 vmax=top_cluster_connections['count'].max())


def get_weight(count):
    return np.interp(count,
                    (top_cluster_connections['count'].min(), top_cluster_connections['count'].max()),
                    (2, 20))

for _, row in stations.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color=color_map[row['cluster_id']] if row['cluster_id'] != -1 else 'lightgray',
        fill=True,
        fill_opacity=0.7,
        popup=f"Station: {row['station_name']}<br>Cluster: {row['cluster_id']}"
    ).add_to(cluster_connection_map)

# Print all bike routes
for index, row in top_cluster_connections.iterrows():
    
    # popup_content = f"""
    # {row['start_station']} → {row['end_station']}<br>
    # Count: {row['count']}
    # """

    folium.PolyLine(
        locations=[[row['start_lat'], row['start_lon']],
                   [row['end_lat'], row['end_lon']]],
                   weight=get_weight(row['count']),
                   opacity=0.5,
                   color=color_scale(row['count']),
                #    popup=folium.Popup(popup_content, max_width=300),
                    ).add_to(cluster_connection_map)

used_clusters = set(top_cluster_connections['cluster_a']).union(
                 set(top_cluster_connections['cluster_b']))

active_clusters = clusters[clusters['cluster_id'].isin(used_clusters)]

for _, row in active_clusters.iterrows():
    cid = row['cluster_id']
    folium.CircleMarker(
        location=[row['cluster_lat'], row['cluster_lon']],
        radius=10,
        color=color_map[cid],
        fill=False,
        fill_color=color_map[cid],
        fill_opacity=0.4,
        weight=2,
        popup=f"Cluster {cid}<br>Size: {row['cluster_size']}"
    ).add_to(cluster_connection_map)
    

    
cluster_connection_map

### 6.4 Structural Analysis

#### 6.4.1 Network Analysis with NetworkX

In [ ]:
import networkx as nx

# Create directional graph
G = nx.DiGraph()

# Add stations as nodes to graph
for _, station in stations.iterrows():
    G.add_node(station['station_id'],
               name=station['station_name'],
               lat=station['lat'],
               lon=station['lon'],
               elevation=station['elevation'])

# Add trips between stations as edges, with frequency of route as weight
trip_counts = trips.groupby(['start_station_id', 'end_station_id']).size().reset_index(name='weight')
trip_counts = trip_counts.sort_values('weight', ascending=False)#.head(100)

for _, trip in trip_counts.iterrows():
    G.add_edge(trip['start_station_id'],
               trip['end_station_id'],
               weight=trip['weight'])

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

In [ ]:
# Calculate metrics
importers = dict(G.in_degree(weight='weight'))
exporters = dict(G.out_degree(weight='weight'))
total_activity = dict(G.degree(weight='weight'))
betweenness = nx.betweenness_centrality(G, weight='weight') # How often a node sits on the shortest path between other nodes. Acts as a bridge
pagerank = nx.pagerank(G, weight='weight') # Network hubs

In [ ]:
# Visualize network in Folium
n_edges = 1000

network_map = folium.Map(location=oslo_coordinates, zoom_start=13, tiles='CartoDB positron')

# Add edges to map
top_edges = sorted(
    G.edges(data=True),
    key=lambda x: x[2]['weight'],
    reverse=True
    )[:n_edges]

# Compute weights for scaling
weights = [w['weight'] for _, _, w in top_edges]
min_weight, max_weight = min(weights), max(weights)

colormap = cm.LinearColormap(['blue', 'darkblue', 'navy'],
                              vmin=min_weight, vmax=max_weight)

def scale(weight, val_min, val_max, scale_min, scale_max):
    return np.interp(
        weight,
        (val_min, val_max),
        (scale_min, scale_max))

    
for e1, e2, data in top_edges:
    lat1, lon1 = G.nodes[e1]['lat'], G.nodes[e1]['lon']
    lat2, lon2 = G.nodes[e2]['lat'], G.nodes[e2]['lon']
    thickness = scale(data['weight'], min_weight, max_weight, 1, 26)

    folium.PolyLine(
        locations=[[lat1, lon1], [lat2, lon2]],
        weight=thickness,
        opacity=0.1,#scale(data['weight'], min_weight, max_weight, 0.3, 0.8),
        color=colormap(data['weight']),#'blue',
        dash_array='5, 5' if thickness < 2 else None
        
    ).add_to(network_map)


# Add nodes to map
for node, data in G.nodes(data=True):
    popup_content = "popup"

    folium.CircleMarker(
        location=[data['lat'], data['lon']],
        color='gray',
        radius=2,
        fill=True,
        fill_opacity=0.3,
        popup=popup_content
    ).add_to(network_map)


# Visualize top hubs
top_nodes = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:30]

for node, score in top_nodes:
    lat, lon = G.nodes[node]['lat'], G.nodes[node]['lon']
    size = scale(score, min(pagerank.values()), max(pagerank.values()), 3, 10)
    folium.CircleMarker(
        location=[lat, lon],
        radius=size,
        color='#DAA520',
        fill=True,
        fill_opacity=0.8,
    ).add_to(network_map)

network_map

In [ ]:
# plt.figure(figsize=(15,10))
# trips["hour"] = pd.to_datetime(trips["started_at"]).dt.hour
# sns.lineplot(data=trips[trips['slope_type']!="flat"], x="hour", y="duration", hue="slope_type", estimator="mean")

In [ ]:
# weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
# trips["weekday"] = pd.to_datetime(trips["started_at"]).dt.day_name()
# sns.countplot(data=trips, x="weekday", order=weekday_order)
# plt.title("Number of Trips by Weekday")

Most people assume weekends are peak for casual biking, but data shows otherwise. Turns out, weekday commuting is the real engine behind Bysykkel usage  — not leisure rides.

⸻  

Quick Summary :  
	•	Sunday = lowest ride count  
	•	Saturday = next lowest  
	•	Weekdays = consistently higher  
	•	Likely interpretation: Bysykkel is primarily a commuter tool, not just a weekend toy.  
  


In [ ]:
# sns.barplot(data=trips, x="weekday", y="duration", order=weekday_order, estimator="mean")
# plt.title("Average Trip Duration by Weekday")

In [ ]:
# sns.barplot(data=trips, x="weekday", y="distance", order=weekday_order, estimator="sum")
# plt.title("Total Trip Duration by Weekday")

🚴‍♂️ Weekends = fewer trips, but longer ones  
💼 Weekdays = more trips, but shorter  

What that suggests:  
	•	Weekdays: People are riding to work, school, or errands — short, practical, repeatable trips.  
	•	Weekends: People ride for leisure — longer joyrides, detours, exploration, maybe even group rides.  

#con.close()